In [20]:
import ollama
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='tinyllama', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

The sky blue color you see in the sky is due to the reflection of the sun's light on water droplets or ice crystals in the atmosphere. The specific shade of blue you observe depends on the color composition of the clouds, including the presence or absence of various types of colors such as greenish, yellowish, and purplish. These colors are created through absorption by different types of light-absorbing molecules within the atmosphere. For instance, blue-violet light is absorbed most strongly by water droplets, while red and orange light is most commonly reflected due to their shorter wavelengths and lower absorption strength compared to blue-violet. The exact colors seen in the sky are dependent on atmospheric conditions and vary depending on the location, time of day or year, and season.
The sky blue color you see in the sky is due to the reflection of the sun's light on water droplets or ice crystals in the atmosphere. The specific shade of blue you observe depends on the color com

In [21]:
# Cell 1: detect OS, list & launch functions

import platform, os, subprocess

SYSTEM = platform.system()  # "Darwin", "Linux" or "Windows"

def list_apps():
    """
    Returns a list of {"name":…, "cmd":[…], "shell":BOOL} for each installed app.
    """
    apps = []
    if SYSTEM == "Darwin":
        apps_dir = "/Applications"
        for item in os.listdir(apps_dir):
            if item.endswith(".app"):
                path = os.path.join(apps_dir, item)
                apps.append({
                    "name": item[:-4],
                    "cmd": ["open", "-a", path],
                    "shell": False
                })
    elif SYSTEM == "Linux":
        desktop_dirs = [
            "/usr/share/applications",
            os.path.expanduser("~/.local/share/applications")
        ]
        for d in desktop_dirs:
            if os.path.isdir(d):
                for f in os.listdir(d):
                    if f.endswith(".desktop"):
                        name = f[:-8]
                        # use gtk-launch (needs to be installed)
                        apps.append({
                            "name": name,
                            "cmd": ["gtk-launch", name],
                            "shell": False
                        })
    elif SYSTEM == "Windows":
        start_dirs = [
            os.path.join(os.getenv("PROGRAMDATA", r"C:\ProgramData"),
                         r"Microsoft\Windows\Start Menu\Programs"),
            os.path.join(os.getenv("APPDATA", ""),
                         r"Microsoft\Windows\Start Menu\Programs")
        ]
        for d in start_dirs:
            if os.path.isdir(d):
                for root, _, files in os.walk(d):
                    for f in files:
                        if f.lower().endswith(".lnk"):
                            name = f[:-4]
                            path = os.path.join(root, f)
                            apps.append({
                                "name": name,
                                # we'll use os.startfile on this .lnk
                                "cmd": [path],
                                "shell": False
                            })
    else:
        raise NotImplementedError(f"Unsupported OS: {SYSTEM}")
    return sorted(apps, key=lambda x: x["name"].lower())

def launch_app(app):
    """
    Given one of the dicts from list_apps, actually run it.
    """
    if SYSTEM == "Windows":
        lnk = app["cmd"][0]
        try:
            os.startfile(lnk)
            print(f"✅ Launched {app['name']}")
        except Exception as e:
            print(f"❌ Failed to launch {app['name']}: {e}")
    else:
        try:
            subprocess.run(app["cmd"], check=True, shell=app["shell"])
            print(f"✅ Launched {app['name']}")
        except Exception as e:
            print(f"❌ Failed to launch {app['name']}: {e}")


In [22]:
# Cell 2 (updated): list, prompt, and open by number or name

apps = list_apps()
if not apps:
    print("⚠️  No applications found on your system.")
else:
    # 1) Print the list
    for i, app in enumerate(apps, 1):
        print(f"{i}. {app['name']}")
    # 2) Prompt
    choice = input("\nEnter the NUMBER or EXACT NAME of the app to open: ").strip()
    
    # 3) Try number first
    launched = False
    if choice.isdigit():
        idx = int(choice) - 1
        if 0 <= idx < len(apps):
            launch_app(apps[idx])
            launched = True
    # 4) If not a valid number, try exact name match
    if not launched:
        for app in apps:
            if app['name'].lower() == choice.lower():
                launch_app(app)
                launched = True
                break

    if not launched:
        print("❌ Invalid selection. Please enter one of the listed numbers or exact app names.")


1. AI Studio 2025.0.0
2. Amphetamine
3. Asana
4. ChatGPT
5. Claude
6. Compressor
7. Cursor
8. Discord
9. djay Pro
10. Docker
11. Elgato Control Center
12. Elgato Stream Deck
13. GitHub Desktop
14. Google Chrome
15. Google Docs
16. Google Drive
17. Google Sheets
18. Google Slides
19. Logi Options
20. logioptionsplus
21. Microsoft Excel
22. Microsoft OneNote
23. Microsoft PowerPoint
24. Microsoft Teams
25. Microsoft Word
26. Mos
27. MySQLWorkbench
28. Ollama
29. Ollamac
30. OneDrive
31. R
32. Remote for Mac
33. RStudio
34. Safari
35. The Unarchiver
36. Unzip - RAR ZIP 7Z Unarchiver
37. Visual Studio Code
38. WinZip
39. zoom.us
✅ Launched ChatGPT


In [27]:
# Cell 3: JSON‐only chooser + fallback keyword match

import json, re
from ollama import chat, ChatResponse

def choose_app_and_launch():
    # 1) Ask the user what they want
    user_request = input("🤖 What should I open? ").strip()
    
    # 2) Build the dynamic list of apps
    list_str = "\n".join(f"{i}. {app['name']}" for i, app in enumerate(apps, 1))
    
    # 3) Prompt TinyLlama to return ONLY a JSON object
    prompt = f"""
You are given a numbered list of applications:
{list_str}

When the user says: "{user_request}"
Respond with exactly one JSON object and nothing else, in this format:

{{"index": N}}

where N is the number (1–{len(apps)}) of the app to open.
"""
    
    resp: ChatResponse = chat(
        model="tinyllama",
        messages=[
            {"role": "system", "content": "Return ONLY the JSON."},
            {"role": "user",   "content": prompt}
        ]
    )
    text = resp.message.content.strip()
    print(f"Model replied ▶️ {text!r}")
    
    # 4) Try to extract the first JSON object
    m = re.search(r"\{.*?\}", text, flags=re.S)
    idx = None
    if m:
        try:
            obj = json.loads(m.group(0))
            idx = int(obj.get("index", 0)) - 1
        except Exception:
            idx = None
    
    # 5) Fallback: keyword‐match against your apps list
    if idx is None or not (0 <= idx < len(apps)):
        print("⚠️  JSON parse failed or out of range, falling back to keyword match…")
        low = user_request.lower()
        for i, app in enumerate(apps):
            if app["name"].lower() in low:
                idx = i
                print(f"✅ Matched by keyword: '{app['name']}' → {i+1}")
                break
    
    # 6) Launch or error
    if idx is None or idx < 0 or idx >= len(apps):
        print("❌ Could not determine which app to open.")
    else:
        launch_app(apps[idx])

# Run it:
choose_app_and_launch()


Model replied ▶️ 'To handle the user\'s command "open google slide", where N is the desired index of an application to open, we can use a loop that iterates through each app in the list and extracts its corresponding JSON object. Each time it encounters a matching index, it will return just the object, leaving other non-matching indexes empty:\n\n```javascript\nconst apps = ["Amiphaime", "ChaetzGPT", "Claude", "Discord", "djayPro", "Docker", "ElgaimControlCenter", "Elgaismaplus", "Google Docs", "Google Drive", "Google Sheets", "logiOptions", "logioptionpluss", "Microsoft Excel", "Microsoft OneNote", "Microsoft PowerPoint", "Microsoft Teams", "Microsoft Word", "Mos", "MySQLWorkbench", "Olmac", "OneDrive", "R", "Remote for Mac", "RStudio", "Safari", "The UnaRChiver", "Unzip - RAW ZIP 7Z UnaRChiver", "Visual Studio Code", "WinZip", "zoom.us"];\nconst appIndex = Number(process.argv[2]);\nif (apps.includes(appIndex)) {\n  const jsonObj = apps.filter((index, app) => index === appIndex)[0];\n